In [6]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

from utils import helper, config, rayer

import ray
import time
from ml.models.base.v2.brisk_xgboost import BriskXGBoost
from ml.models.base.v2.slug_xgboost import SlugXGBoost
from ml.models.base.slug_ann import SlugANN
from ml.models.base.slug_rf import SlugRF
from ml.models.base.slug_knn import SlugKNN
from ml.models.base.v2.brisk_bagging import BriskBagging

from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common
from sklearn.preprocessing import StandardScaler


import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150) # print: change width of display
pd.options.display.max_colwidth = 100 # print: max width column
import warnings
warnings.filterwarnings('ignore')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
rayer.get_local_cluster(num_cpus=4)

<module 'ray' from '/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-275852-64-wasif-dev-ray/lib/python3.9/site-packages/ray/__init__.py'>

In [7]:
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-17 13:47:48.345999 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 2 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/5.0 CPU
 0.00/16.764 GiB memory
 0.00/4.898 GiB object_store_memory

Demands:
 {'CPU': 1}: 4+ from request_resources()


In [8]:
df = helper.get_tokamat_dataset()
df = df.reset_index()

/home/wasif/python-asd/xai/notebooks/fusion/../../utils


In [9]:
df = common.label_encode(df)
df = df.fillna(-1)

In [41]:
# TAUTH * TAUC93, Estimated thermal energy confinement time in seconds * Correction factor for thermal confinement time
# WTOT Estimated total plasma energy content in Joules.
# WTH Estimated thermal plasma energy content in Joules.
# PL Estimated Loss Power not corrected for charge exchange and unconfined orbit losses in watts.
# PLTH Estimated Loss Power corrected for charge exchange and unconfined orbit losses in Watts

In [10]:
df.columns

Index(['index', 'TOK_ID', 'DIVNAME', 'LCUPDATE', 'DATE', 'SHOT', 'TIME', 'TIME_ID', 'AUXHEAT', 'PHASE', 'ISEQ', 'MEFF', 'PGASA', 'PGASZ', 'BGASA',
       'BGASZ', 'BGASA2', 'BGASZ2', 'PELLET', 'XGASA', 'XGASZ', 'CONFIG', 'RGEO', 'AMIN', 'KAPPA', 'KAPPAA', 'KAREA', 'DELTA', 'INDENT', 'AREA',
       'VOL', 'SURFFORM', 'SEPLIM', 'WALMAT', 'DIVMAT', 'LIMMAT', 'EVAP', 'BT', 'IP', 'VSURF', 'Q95', 'NEL', 'NELFORM', 'DNELDT', 'POHM', 'ENBI',
       'PINJ', 'BSOURCE', 'PINJ2', 'BSOURCE2', 'PNBI', 'PFLOSS', 'PECRHC', 'PECRH', 'ICFREQ', 'ICSCHEME', 'ICANTEN', 'PICRHC', 'PICRH', 'PALPHA',
       'DWHC', 'WFFORM', 'ICFORM', 'WTOT', 'WTH', 'PL', 'PLTH', 'TAUTOT', 'TAUTH', 'TAUC92', 'TAUC93', 'TOK'],
      dtype='object')

In [11]:
df.head()

,index,TOK_ID,DIVNAME,LCUPDATE,DATE,SHOT,TIME,TIME_ID,AUXHEAT,PHASE,ISEQ,MEFF,PGASA,PGASZ,BGASA,BGASZ,BGASA2,BGASZ2,PELLET,XGASA,XGASZ,CONFIG,RGEO,AMIN,KAPPA,KAPPAA,KAREA,DELTA,INDENT,AREA,VOL,SURFFORM,SEPLIM,WALMAT,DIVMAT,LIMMAT,EVAP,BT,IP,VSURF,Q95,NEL,NELFORM,DNELDT,POHM,ENBI,PINJ,BSOURCE,PINJ2,BSOURCE2,PNBI,PFLOSS,PECRHC,PECRH,ICFREQ,ICSCHEME,ICANTEN,PICRHC,PICRH,PALPHA,DWHC,WFFORM,ICFORM,WTOT,WTH,PL,PLTH,TAUTOT,TAUTH,TAUC92,TAUC93,TOK
0,0,0,8,19940125,19820622,5980,0.959961,960,6,12,70,2.0,2.0,0,0,0,0,0,5,0.0,0,0,1.667969,0.384033,1.041992,1.041992,1.041992,0.0,0.0,0.482666,5.058594,25.828125,0.208252,5,9,4,9,2.203125,293900,1.073242,3.740234,-9223372036854775808,0,6404,315400,0.0,0,0,0,0,0,0,0,0,0,5,1,0,0,0,0,0,0,27950,27950,315400,315400,0.088623,0.088623,1.000000,1.000000,0
1,1,0,8,19940125,19820622,5980,1.009766,1010,6,12,70,2.0,2.0,0,0,0,0,0,5,0.0,0,0,1.667969,0.383789,1.041992,1.041992,1.041992,0.0,0.0,0.482666,5.058594,25.812500,0.208252,5,9,4,9,2.203125,294300,1.106445,3.734375,-9223372036854775808,0,7220,325500,0.0,0,0,0,0,0,0,0,0,0,5,1,0,0,0,0,0,0,28610,28610,325500,325500,0.087891,0.087891,1.000000,1.000000,0
2,2,0,8,19940125,19820622,5980,1.059570,1060,6,12,70,2.0,2.0,0,0,0,0,0,5,0.0,0,0,1.667969,0.384033,1.041992,1.041992,1.041992,0.0,0.0,0.482666,5.058594,25.828125,0.208496,5,9,4,9,2.203125,294400,1.015625,3.734375,-9223372036854775808,0,646,299200,0.0,0,0,0,0,0,0,0,0,0,5,1,0,0,0,0,0,0,28860,28860,299200,299200,0.096436,0.096436,1.000000,1.000000,0
3,3,0,8,19940125,19820622,5980,1.185547,1186,2,0,70,1.5,2.0,0,1,1,0,0,5,0.0,0,0,1.698242,0.406494,0.984375,0.984375,0.983887,0.0,0.0,0.510742,5.449219,27.031250,0.155640,5,9,4,9,2.205078,298000,-0.069824,4.343750,-9223372036854775808,0,3085,0,41984.0,2620000,2,0,0,2596000,570700,0,0,0,5,1,0,0,0,1,23430,0,103500,80110,1323000,752300,0.078247,0.106506,0.833496,0.833496,0
4,4,0,8,19940125,19820622,5980,1.205078,1205,2,3,70,1.5,2.0,0,1,1,0,0,5,0.0,0,0,1.694336,0.410156,0.975098,0.975098,0.974609,0.0,0.0,0.515137,5.480469,27.093750,0.156372,5,9,4,9,2.205078,295900,0.232056,4.562500,-9223372036854775808,0,670,68660,41984.0,2620000,2,0,0,2599000,585700,0,0,0,5,1,0,0,0,1,24260,0,119600,95370,2456000,1870000,0.048706,0.050995,0.666504,0.666504,0


In [22]:
import numpy as np
df[df['ENBI']==np.inf]

,index,TOK_ID,DIVNAME,LCUPDATE,DATE,SHOT,TIME,TIME_ID,AUXHEAT,PHASE,ISEQ,MEFF,PGASA,PGASZ,BGASA,BGASZ,BGASA2,BGASZ2,PELLET,XGASA,XGASZ,CONFIG,RGEO,AMIN,KAPPA,KAPPAA,KAREA,DELTA,INDENT,AREA,VOL,SURFFORM,SEPLIM,WALMAT,DIVMAT,LIMMAT,EVAP,BT,IP,VSURF,Q95,NEL,NELFORM,DNELDT,POHM,ENBI,PINJ,BSOURCE,PINJ2,BSOURCE2,PNBI,PFLOSS,PECRHC,PECRH,ICFREQ,ICSCHEME,ICANTEN,PICRHC,PICRH,PALPHA,DWHC,WFFORM,ICFORM,WTOT,WTH,PL,PLTH,TAUTOT,TAUTH,TAUC92,TAUC93,TOK
1603,1603,7,1,20180403,19990225,11652,3.349609,3350,2,3,70,2.0,2.0,0,2,1,2,1,5,0.0,0,4,1.656250,0.486084,1.824219,1.657227,1.636719,0.170166,0.0,1.230469,12.648438,46.75000,0.044128,0,1,1,2,-2.205078,800700,0.122925,4.671875,-9223372036854775808,0,5624,98460,inf,4983000,55,5111000,622809,10020000,1077000,0,0,0,5,1,0,0,0,0,269500,0,668000,398400,10100000,8980000,0.065918,0.051941,1.0,1.0,1
1604,1604,7,1,20180403,19990225,11653,3.150391,3150,2,3,70,2.0,2.0,0,2,1,2,1,5,0.0,0,4,1.654297,0.487061,1.814453,1.650391,1.629883,0.171143,0.0,1.230469,12.632812,46.59375,0.045837,0,1,1,2,-2.205078,800200,0.118896,4.667969,-9223372036854775808,0,3499,95220,inf,4992000,55,5106000,622809,10020000,1075000,0,0,0,5,1,0,0,0,0,274300,0,680000,405700,10200000,9030000,0.066772,0.052704,1.0,1.0,1
1652,1652,7,1,20180403,19990319,11801,3.300781,3300,2,7,70,2.0,2.0,0,2,1,2,1,2,0.0,0,4,1.642578,0.524902,1.778320,1.585938,1.553711,0.294189,0.0,1.372070,13.882812,49.12500,0.037872,0,1,1,2,-2.058594,999100,0.395264,4.003906,-9223372036854775808,0,5231,394900,inf,2490000,60,2506000,622809,4996000,585700,0,0,0,5,1,0,0,0,0,28630,0,543000,514700,5360000,4770000,0.101990,0.110291,1.0,1.0,1
1653,1653,7,1,20180403,19990319,11801,4.718750,4720,2,7,70,2.0,2.0,0,2,1,2,1,2,0.0,0,4,1.641602,0.526367,1.769531,1.575195,1.543945,0.282227,0.0,1.371094,13.867188,49.06250,0.037689,0,1,1,2,-2.056641,997600,0.546875,3.968750,-9223372036854775808,0,871,545700,inf,2485000,60,4892000,622809,7375000,934400,0,0,0,5,1,0,0,0,0,33590,0,427000,393900,8480000,7540000,0.050690,0.053558,1.0,1.0,1
1654,1654,7,1,20180403,19990319,11803,3.050781,3050,2,7,70,2.0,2.0,0,2,1,2,1,2,0.0,0,4,1.642578,0.524414,1.780273,1.587891,1.555664,0.292969,0.0,1.372070,13.882812,49.12500,0.037781,0,1,1,2,-2.058594,999800,0.393555,3.996094,-9223372036854775808,0,3379,393400,inf,2490000,60,2496000,622809,4985000,582400,0,0,0,5,1,0,0,0,0,28900,0,547000,517700,5350000,4760000,0.101990,0.111328,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8277,8277,6,15,20070531,20060418,119996,0.620117,620,2,0,70,2.0,2.0,0,2,1,0,0,5,0.0,0,4,0.863281,0.593750,2.435547,2.302734,2.115234,0.804199,0.0,2.550781,12.703125,37.65625,0.084351,0,1,4,9,0.439697,999896,0.231201,8.640625,-9223372036854775808,0,3624,381837,inf,6136190,3,0,0,6127800,334389,0,0,0,5,1,0,0,0,0,38921,0,289416,231857,6012070,5677680,0.048126,0.038605,1.0,1.0,6
8278,8278,6,15,20070531,20060418,120001,0.799805,800,2,0,70,2.0,2.0,0,2,1,0,0,5,0.0,0,4,0.872559,0.598633,2.433594,2.324219,2.126953,0.666992,0.0,2.617188,13.132812,38.34375,0.088562,0,1,4,9,0.435303,1025140,0.325195,7.832031,-9223372036854775808,0,4945,392460,inf,6198590,3,0,0,6183040,868196,0,0,0,5,1,0,0,0,0,36748,0,332297,269149,6267300,5399110,0.053009,0.046295,1.0,1.0,6
8279,8279,6,15,20070531,20060418,120003,0.700195,700,2,3,70,2.0,2.0,0,2,1,0,0,5,0.0,0,4,0.857910,0.616699,2.269531,2.111328,1.946289,0.703613,0.0,2.523438,12.539062,36.62500,0.056244,0,1,4,9,0.439453,1019960,0.341309,7.781250,-9223372036854775808,0,5679,417173,inf,6165910,3,0,0,6186930,531645,0,0,0,5,1,0,0,0,0,36106,0,294787,221722,6726300,6194650,0.043823,0.035278,1.0,1.0,6
8280,8280,6,15,20070531,20060418,120005,0.620117,620,2,3,70,2.0,2.0,0,2,1,0,0,5,0.0,0,4,0.863770,0.625000,2.183594,2.042969,1.869141,0.731934,0.0,2.503906,12.437500,36.18750,0.05374

In [13]:
potential_targets = ['WTOT', 'WTH', 'PLTH']
y = df[potential_targets[0]]

X = df[df.columns[~df.columns.isin(potential_targets)]]
X = X.drop(['TOK_ID', 'LCUPDATE', 'DATE', 'NEL', 'ENBI'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [14]:
X_train.head()

,index,DIVNAME,SHOT,TIME,TIME_ID,AUXHEAT,PHASE,ISEQ,MEFF,PGASA,PGASZ,BGASA,BGASZ,BGASA2,BGASZ2,PELLET,XGASA,XGASZ,CONFIG,RGEO,AMIN,KAPPA,KAPPAA,KAREA,DELTA,INDENT,AREA,VOL,SURFFORM,SEPLIM,WALMAT,DIVMAT,LIMMAT,EVAP,BT,IP,VSURF,Q95,NELFORM,DNELDT,POHM,ENBI,PINJ,BSOURCE,PINJ2,BSOURCE2,PNBI,PFLOSS,PECRHC,PECRH,ICFREQ,ICSCHEME,ICANTEN,PICRHC,PICRH,PALPHA,DWHC,WFFORM,ICFORM,PL,TAUTOT,TAUTH,TAUC92,TAUC93,TOK
9523,9523,15,74159,2.970703,2970,5,13,70,2.0,2.0,0,2,1,0,0,5,20.1875,10,2,1.750000,0.459961,1.000000,1.000000,1.000000,0.000000,0.0,0.665039,7.308594,31.781250,0.000000,2,6,1,0,2.230469,416000,0.674805,2.701172,0,2568,280800,48448.0,1220000,43,0,0,1220000,16880,0,0,32500000,4,0,542100,542100,0,0,9858,0,2043000,0.061951,0.057587,1.0,1.0,11
7540,7540,15,45262,0.669922,670,2,0,74,1.0,1.0,0,1,1,0,0,5,0.0000,0,3,1.321289,0.264893,1.355469,1.319336,1.319336,0.337646,0.0,0.291016,2.416016,16.453125,0.070740,5,1,1,12,1.238281,139500,0.275391,4.253906,0,2737,38400,32128.0,1617000,81,0,0,1406000,402000,0,0,0,5,1,0,0,0,0,8516,0,1440000,0.013657,0.010750,1.0,1.0,4
8008,8008,15,62060,0.890137,890,2,7,52,1.0,1.0,0,1,1,0,0,5,0.0000,0,3,1.313477,0.263184,1.328125,1.331055,1.331055,0.253662,0.0,0.289795,2.392578,16.046875,0.072571,5,1,1,12,1.283203,217300,0.938965,2.816406,0,4759,204000,32208.0,1224000,81,0,0,1213000,151900,0,0,0,5,1,0,0,0,0,4236,0,1390000,0.024124,0.023666,1.0,1.0,4
3022,3022,5,27226,2.250000,2250,3,3,101,2.0,2.0,0,2,1,2,1,2,0.0000,0,4,1.646484,0.480957,1.885742,1.712891,1.682617,0.261963,0.0,1.245117,12.640625,47.187500,0.069214,6,10,5,2,-2.523438,998400,0.149170,4.507812,0,6143,279100,inf,2421000,66,4993000,622808,7277000,419100,2719000,2719000,0,5,1,0,0,0,0,197600,0,10300000,0.071228,0.054688,1.0,1.0,1
2664,2664,3,19314,6.199219,6200,2,3,53,2.0,2.0,0,2,1,2,1,5,20.1875,10,4,1.646484,0.511719,1.762695,1.586914,1.557617,0.228760,0.0,1.305664,13.257812,47.687500,0.044769,6,2,2,2,-1.993164,999700,0.217896,3.601562,0,7385,255800,inf,1725000,66,2535000,622808,4191000,148900,0,0,30000000,4,0,6112000,4890000,0,0,115200,0,9230000,0.082092,0.073120,1.0,1.0,1


In [16]:
X_train.max().sort_values()

INDENT     2.054443e-01
SEPLIM     2.663574e-01
DELTA      8.481445e-01
NELFORM    1.000000e+00
ICFORM     1.000000e+00
               ...     
PL         2.616000e+07
PINJ       2.790000e+07
ICFREQ     8.000000e+07
SHOT       9.602140e+08
ENBI                inf
Length: 65, dtype: float64

In [48]:
# X_train_scalar = StandardScaler().fit_transform(X_train.copy())
# X_test_scalar = StandardScaler().fit_transform(X_test.copy())


In [53]:


brisk_bagging_1 = BriskBagging('brisk_bagging_1')
brisk_bagging_1.boosted_round = 100
brisk_bagging_1.n_trials = 50



In [54]:
brisk_bagging_1.fetch_model(X_train, X_test, y_train, y_test)

2022-11-29T10:40:10CST : INFO : brisk_bagging : __discover_model__ : 94 : Message : brisk_bagging_1: Starting training for trials:50, n_estimators  30
[W 2022-11-29 10:40:10,705] Trial 0 failed because of the following error: ValueError("Input X contains infinity or a value too large for dtype('float32').")
Traceback (most recent call last):
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-275852-64-wasif-dev-ray/lib/python3.9/site-packages/optuna/_optimize.py", line 189, in _run_trial
    value = func(trial)
  File "/home/wasif/python-asd/xai/notebooks/fusion/../../ml/models/base/v2/brisk_bagging.py", line 108, in <lambda>
    obj_func = lambda trial: self.__objective__(trial, X_train, X_test, y_train, y_test)
  File "/home/wasif/python-asd/xai/notebooks/fusion/../../ml/models/base/v2/brisk_bagging.py", line 76, in __objective__
    model.fit(X_train, y_train.values.ravel())
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a

ValueError: Input X contains infinity or a value too large for dtype('float32').

In [16]:

slug_xgb = SlugXGBoost('slug_xgb', X_train_scalar, X_test_scalar, y_train, y_test)
slug_xgb.boosted_round = 50
slug_xgb.n_trials = 1000


In [17]:
slug_xgb.fetch_model()

2022-11-17T13:52:14CST : INFO : slug_xgboost : __discover_model__ : 168 : Message : slug_xgb: Starting training for trials:1000, boosted rounds: 50, max depth: 10
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 181 : Message : slug_xgb: Number of trials: 1000
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 183 : Message : Best trial:611
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 185 : Message :   Params: 
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     lambda 0.019399866419368653
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     alpha 0.0020906308336665715
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     eta 0.3727957591519319
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     gamma 0.0033926747144862554
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ :

In [36]:

slug_rf = SlugRF('slug_rf', X_train_scalar, X_test_scalar, y_train, y_test)
slug_rf.max_n_estimators = 5000
slug_rf.n_trials = 100
slug_rf.max_depth = 200

In [37]:
best_fit = slug_rf.fetch_model()

2022-11-17T14:56:36CST : INFO : slug_rf : __discover_model__ : 105 : Message : slug_rf: Starting training for trials:100, max depth: 200, max estimators 5000
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 122 : Message : slug_rf: Number of trials: 100
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 124 : Message : Best trial:74
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 126 : Message :   Params: 
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     n_estimators 62
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     max_depth 26
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     min_samples_split 52
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     min_samples_leaf 34
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 142 : Message :   test r2 score: 0.3992977208659255
2022-11-17T14:59:59CST : INFO : 

In [35]:
slug_rf.score

[0.5264056467634264, 0.39565984792823017]